# Rag Engine

<span style="color:red">**YOU NEED TO UPDATE YOUR PROJECT_ID AND LOCATION**</span>

In [2]:
import os
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine
from vertexai.preview import rag
import vertexai

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
VERTEX_AI_SEARCH_LOCATION = "global"
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Configuration

- Create a new Cloud Storage bucket with PDF.
- Datastore name MUST BE LOWERCASE

In [7]:
# The datastore name can only contain lowercase letters, numbers, and hyphens
DATASTORE_NAME = "2025-adk-workshop-policy-tobedeleted"
DATASTORE_ID = f"{DATASTORE_NAME}-id"

GCS_BUCKET = "gs://2025-adk-workshop-policy"


# Part 1 - Datastore (Run once only)

### Create and Populate a Datastore

RAG is not supported for the Vertex AI Search DataStore with 'chunking config'.

In [8]:
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        content_config=discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
    )

    operation = client.create_data_store(
        request=discoveryengine.CreateDataStoreRequest(
            parent=client.collection_path(project_id, location, "default_collection"),
            data_store=data_store,
            data_store_id=data_store_id,
        )
    )

    # Make the request
    response = operation.result(timeout=90)
    return response.name

In [9]:
def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: str, # Bucket path
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    source_documents = [f"{gcs_uri}/*"]

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=source_documents, data_schema="content"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )

    # Make the request
    operation = client.import_documents(request=request)

    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    return operation.operation.name

In [10]:
create_data_store(PROJECT_ID, VERTEX_AI_SEARCH_LOCATION, DATASTORE_NAME, DATASTORE_ID)

'projects/671247654914/locations/global/collections/default_collection/dataStores/2025-adk-workshop-policy-tobedeleted-id'

Next is to import the Cloud Storage document, this steps takes about 10 minutes.

In [11]:
import_documents(PROJECT_ID, VERTEX_AI_SEARCH_LOCATION, DATASTORE_ID, GCS_BUCKET)

'projects/671247654914/locations/global/collections/default_collection/dataStores/2025-adk-workshop-policy-tobedeleted-id/branches/0/operations/import-documents-6174108388888858925'

## Part 2 - Create a Search Engine (Run once only)

In [12]:
def create_engine(
    project_id: str, location: str, engine_name: str, engine_id: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    engine = discoveryengine.Engine(
        display_name=engine_name,
        solution_type=discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        data_store_ids=[data_store_id],
        search_engine_config=discoveryengine.Engine.SearchEngineConfig(
            search_tier=discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE,
        ),
    )
    print(engine.display_name)

    request = discoveryengine.CreateEngineRequest(
        parent=client.collection_path(project_id, location, "default_collection"),
        engine=engine,
        engine_id=engine_id,
    )

    # Make the request
    operation = client.create_engine(request=request)
    response = operation.result(timeout=90)
    return response.name

In [13]:
DATASTORE_NAME, DATASTORE_ID

('2025-adk-workshop-policy-tobedeleted',
 '2025-adk-workshop-policy-tobedeleted-id')

In [14]:
ENGINE_NAME = DATASTORE_NAME + "-engine"
ENGINE_ID = DATASTORE_ID + "-engine"

response = create_engine(
    project_id=PROJECT_ID,
    location=VERTEX_AI_SEARCH_LOCATION,
    engine_name=ENGINE_NAME,
    engine_id=ENGINE_ID,
    data_store_id=DATASTORE_ID,
)

response

2025-adk-workshop-policy-tobedeleted-engine


'projects/671247654914/locations/global/collections/default_collection/engines/2025-adk-workshop-policy-tobedeleted-id-engine'

## Part 3 - Create corpus (Run only once)

In [15]:
from vertexai.preview import rag

In [16]:
VERTEX_AI_SEARCH_LOCATION, ENGINE_NAME, ENGINE_ID

('global',
 '2025-adk-workshop-policy-tobedeleted-engine',
 '2025-adk-workshop-policy-tobedeleted-id-engine')

In [17]:
"""
Format:
    ``projects/{project}/locations/{location}/collections/{collection}/engines/{engine_id}/servingConfigs/{serving_config}``
or
    ``projects/{project}/locations/{location}/collections/{collection}/dataStores/{data_store_id}/servingConfigs/{serving_config}``
"""

config = f"projects/{PROJECT_ID}/locations/{VERTEX_AI_SEARCH_LOCATION}/collections/default_collection/engines/{ENGINE_ID}/servingConfigs/default_search"
print(config)

vertex_ai_search_config = rag.VertexAiSearchConfig(
    serving_config=config,
)


projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-adk-workshop-policy-tobedeleted-id-engine/servingConfigs/default_search


In [18]:
rag_corpus = rag.create_corpus(
    display_name=ENGINE_NAME,
    vertex_ai_search_config=vertex_ai_search_config,
)


In [19]:
print(rag_corpus.name)

projects/671247654914/locations/us-central1/ragCorpora/3054003497310617600


In [20]:
# Check the corpus just created
new_corpus = rag.get_corpus(name=rag_corpus.name)
new_corpus

RagCorpus(name='projects/671247654914/locations/us-central1/ragCorpora/3054003497310617600', display_name='2025-adk-workshop-policy-tobedeleted-engine', description='', embedding_model_config=EmbeddingModelConfig(publisher_model=None, endpoint=None, model=None, model_version_id=None), vector_db=None, vertex_ai_search_config=VertexAiSearchConfig(serving_config='projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-adk-workshop-policy-tobedeleted-id-engine/servingConfigs/default_search'), backend_config=RagVectorDbConfig(vector_db=None, rag_embedding_model_config=None))

## List corpora

In [21]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/hello-world-418507/locations/us-central1/ragCorpora/1113515007867355136"
  display_name: "2025-test-datastore-name-engine"
  create_time {
    seconds: 1742560106
    nanos: 86801000
  }
  update_time {
    seconds: 1742560106
    nanos: 86801000
  }
  corpus_status {
    state: ACTIVE
  }
  vertex_ai_search_config {
    serving_config: "projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-test-datastore-name-id-engine/servingConfigs/default_search"
  }
}
rag_corpora {
  name: "projects/hello-world-418507/locations/us-central1/ragCorpora/713820540938223616"
  display_name: "Alphabet_10K_2024_corpus"
  description: "Corpus containing Alphabet\'s 10-K 2024 document"
  create_time {
    seconds: 1744590438
    nanos: 956266000
  }
  update_time {
    seconds: 1744590438
    nanos: 956266000
  }
  rag_embedding_model_config {
    vertex_prediction_endpoint {
      endpoint: "projects/hello-world-418507/

# Part 4 - Retrieval

### Query with Rag Engine

You can find the value for `CORPUS_NAME` from the previous cell `name` value.

```json
rag_corpora {
  name: "projects/hello-world-418507/locations/us-central1/ragCorpora/3054003497310617600"
  display_name: "2025-adk-workshop-policy-tobedeleted-engine"
  create_time {
    seconds: 1746602040
    nanos: 531441000
  }
  update_time {
    seconds: 1746602040
    nanos: 531441000
  }
  corpus_status {
    state: ACTIVE
  }
  vertex_ai_search_config {
    serving_config: "projects/hello-world-418507/locations/global/collections/default_collection/engines/2025-adk-workshop-policy-tobedeleted-id-engine/servingConfigs/default_search"
  }
}
```

In [29]:
RETRIEVAL_QUERY = "When can i start taking maternity leave?"
CORPUS_NAME = "projects/hello-world-418507/locations/us-central1/ragCorpora/3054003497310617600"

rag_resource = rag.RagResource(rag_corpus=CORPUS_NAME)
print(rag_resource)

rag_retrieval_config = rag.RagRetrievalConfig(
    top_k=5,  # Optional
    filter=rag.Filter(
        vector_distance_threshold=0.2,  # Optional
    ),
)

response = rag.retrieval_query(
    rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
    text=RETRIEVAL_QUERY,
    similarity_top_k=10,
    # rag_retrieval_config=rag_retrieval_config
)
print(response)


RagResource(rag_corpus='projects/hello-world-418507/locations/us-central1/ragCorpora/3054003497310617600', rag_file_ids=None)
contexts {
  contexts {
    source_uri: "gs://2025-adk-workshop-policy/timeoff-policy.pdf"
    text: "11. When can I start my maternity leave? A: The earliest you can start your leave is <b>28 days before your expected due date</b>. If your baby is premature, your maternity leave will begin on the day your baby is born. 12."
    source_display_name: "timeoff-policy"
  }
  contexts {
    source_uri: "gs://2025-adk-workshop-policy/performance-policy.pdf"
    text: "■ If the active period is substantial, a standard review might occur, focusing on that time."
    source_display_name: "performance-policy"
  }
}



## Query with Gemini


In [26]:
from rich import print as rich_print
from vertexai.preview import rag
from google.genai.types import (
    GenerateContentConfig,
    VertexRagStore,
    Retrieval,
    Tool,
    VertexAISearch,
)
from google import genai


In [30]:

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
MODEL_ID = "gemini-2.0-flash-001"
RETRIEVAL_QUERY = "When can i start taking maternity leave?"


client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

rag_resource = rag.RagResource(
    rag_corpus=CORPUS_NAME,
)

vertex_ai_search_tool = Tool(
    retrieval=Retrieval(vertex_rag_store=VertexRagStore(rag_resources=[rag_resource]))
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=RETRIEVAL_QUERY,
    config=GenerateContentConfig(tools=[vertex_ai_search_tool]),
)

rich_print(response)


GenerateContentResponse(
    candidates=[
        Candidate(
            content=Content(
                parts=[
                    Part(
                        video_metadata=None,
                        thought=None,
                        code_execution_result=None,
                        executable_code=None,
                        file_data=None,
                        function_call=None,
                        function_response=None,
                        inline_data=None,
                        text='You can start your maternity leave as early as 28 days before your expected due date.
If your baby is premature, your maternity leave will begin on the day your baby is born.\n'
                    )
                ],
                role='model'
            ),
            citation_metadata=None,
            finish_message=None,
            token_count=None,
            finish_reason=<FinishReason.STOP: 'STOP'>,
            avg_logprobs=-0.011928290892869998,
            grounding_metadata=GroundingMetadata(
                grounding_chunks=[
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='11. When can I start my maternity leave? A: The earliest you can start your leave
is <b>28 days before your expected due date</b>. If your baby is premature, your maternity leave will begin on the 
day your baby is born. 12.',
                            title='timeoff-policy',
                            uri='gs://2025-adk-workshop-policy/timeoff-policy.pdf'
                        ),
                        web=None
                    ),
                    GroundingChunk(
                        retrieved_context=GroundingChunkRetrievedContext(
                            text='■ If the active period is substantial, a standard review might occur, focusing on
that time.',
                            title='performance-policy',
                            uri='gs://2025-adk-workshop-policy/performance-policy.pdf'
                        ),
                        web=None
                    )
                ],
                grounding_supports=[
                    GroundingSupport(
                        confidence_scores=[0.98975134],
                        grounding_chunk_indices=[0],
                        segment=Segment(
                            end_index=85,
                            part_index=None,
                            start_index=None,
                            text='You can start your maternity leave as early as 28 days before your expected due 
date.'
                        )
                    ),
                    GroundingSupport(
                        confidence_scores=[0.99229574],
                        grounding_chunk_indices=[0],
                        segment=Segment(
                            end_index=174,
                            part_index=None,
                            start_index=86,
                            text='If your baby is premature, your maternity leave will begin on the day your baby 
is born.'
                        )
                    )
                ],
                retrieval_metadata=None,
                retrieval_queries=['When can I start taking maternity leave?'],
                search_entry_point=None,
                web_search_queries=None
            ),
            index=None,
            logprobs_result=None,
            safety_ratings=None
        )
    ],
    create_time=datetime.datetime(2025, 5, 7, 7, 20, 9, 442583, tzinfo=TzInfo(UTC)),
    response_id='qQkbaNeBG5OBm9IPwtOUqQ0',
    model_version='gemini-2.0-flash-001',
    prompt_feedback=None,
    usage_metadata=GenerateContentResponseUsageMetadata(
        cache_tokens_details=None,
        cached_content_token_count=None,
        candidates_token_count=39,
        candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=39)],
        promp

## Cleaning up

Clean up RAG resources created in this notebook.

In [31]:
# rag.delete_corpus(name=CORPUS_NAME)